# Assignment 3

### Part a

In [1]:
# Import relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV, LinearRegression, Ridge, Lasso, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
import patsy
import warnings
from sklearn.base import BaseEstimator, clone
import statsmodels.api as sm
import statsmodels.formula.api as smf
from IPython.display import Markdown
import os
import seaborn as sns
from flaml import AutoML

warnings.simplefilter('ignore')
np.random.seed(1234)

In [2]:
from sklearn.base import TransformerMixin, BaseEstimator
from formulaic import Formula

class FormulaTransformer(TransformerMixin, BaseEstimator):

    def __init__(self, formula, array=False):
        self.formula = formula
        self.array = array

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        df = Formula(self.formula).get_model_matrix(X)
        if self.array:
            return df.values
        return df

In [38]:
def dml(X, D, y, modely, modeld, *, nfolds, classifier=False):
    '''
    DML for the Partially Linear Model setting with cross-fitting

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely: the ML model for predicting the outcome y
    modeld: the ML model for predicting the treatment D
    nfolds: the number of folds in cross-fitting
    classifier: bool, whether the modeld is a classifier or a regressor

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    Dhat: the cross-fitted predictions for the treatment D
    resy: the outcome residuals
    resD: the treatment residuals
    epsilon: the final residual-on-residual OLS regression residual
    '''
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123) # shuffled k-folds
    yhat = cross_val_predict(modely, X, y, cv=cv, n_jobs=-1) # out-of-fold predictions for y
    # out-of-fold predictions for D
    # use predict or predict_proba dependent on classifier or regressor for D
    if classifier:
        Dhat = cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
    else:
        Dhat = cross_val_predict(modeld, X, D, cv=cv, n_jobs=-1)
    # calculate outcome and treatment residuals
    resy = y - yhat
    resD = D - Dhat

    # final stage ols based point estimate and standard error
    point = np.mean(resy * resD) / np.mean(resD**2)
    epsilon = resy - point * resD
    var = np.mean(epsilon**2 * resD**2) / np.mean(resD**2)**2
    stderr = np.sqrt(var / X.shape[0])

    return point, stderr, yhat, Dhat, resy, resD, epsilon

In [4]:
def dml_dirty(X, D, y, modely_list, modeld_list, *,
              stacker=LinearRegression(), nfolds, classifier=False):
    '''
    DML for the Partially Linear Model setting with semi-cross-fitting

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely: the ML model for predicting the outcome y
    modeld: the ML model for predicting the treatment D
    stacker: model used to aggregate predictions of each of the base models
    nfolds: the number of folds in cross-fitting
    classifier: bool, whether the modeld is a classifier or a regressor

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    Dhat: the cross-fitted predictions for the treatment D
    resy: the outcome residuals
    resD: the treatment residuals
    epsilon: the final residual-on-residual OLS regression residual
    '''
    # construct out-of-fold predictions for each model
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)
    yhats = np.array([cross_val_predict(modely, X, y, cv=cv, n_jobs=-1) for modely in modely_list]).T
    if classifier:
        Dhats = np.array([cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
                         for modeld in modeld_list]).T
    else:
        Dhats = np.array([cross_val_predict(modeld, X, D, cv=cv, n_jobs=-1) for modeld in modeld_list]).T
    # calculate stacked residuals by finding optimal coefficients
    # and weigthing out-of-sample predictions by these coefficients
    yhat = stacker.fit(yhats, y).predict(yhats)
    Dhat = stacker.fit(Dhats, D).predict(Dhats)
    resy = y - yhat
    resD = D - Dhat
    # go with the stacked residuals
    point = np.mean(resy * resD) / np.mean(resD**2)
    epsilon = resy - point * resD
    var = np.mean(epsilon**2 * resD**2) / np.mean(resD**2)**2
    stderr = np.sqrt(var / X.shape[0])
    return point, stderr, yhat, Dhat, resy, resD, epsilon

In [5]:
def dr(X, D, y, modely0, modely1, modeld, *, trimming=0.01, nfolds):
    '''
    DML for the Interactive Regression Model setting (Doubly Robust Learning)
    with cross-fitting

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely0: the ML model for predicting the outcome y in the control population
    modely1: the ML model for predicting the outcome y in the treated population
    modeld: the ML model for predicting the treatment D
    trimming: threshold below which to trim propensities
    nfolds: the number of folds in cross-fitting

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    Dhat: the cross-fitted predictions for the outcome D
    resy: the outcome residuals
    resD: the treatment residuals
    drhat: the doubly robust quantity for each sample
    '''
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)
    yhat0, yhat1 = np.zeros(y.shape), np.zeros(y.shape)
    # we will fit a model E[Y| D, X] by fitting a separate model for D==0
    # and a separate model for D==1.
    for train, test in cv.split(X, y):
        # train a model on training data that received treatment zero and predict on all data in test set
        yhat0[test] = clone(modely0).fit(X.iloc[train][D[train]==0], y[train][D[train]==0]).predict(X.iloc[test])
        # train a model on training data that received treatment one and predict on all data in test set
        yhat1[test] = clone(modely1).fit(X.iloc[train][D[train]==1], y[train][D[train]==1]).predict(X.iloc[test])
    # prediction for observed treatment
    yhat = yhat0 * (1 - D) + yhat1 * D
    # propensity scores
    Dhat = cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
    Dhat = np.clip(Dhat, trimming, 1 - trimming)
    # doubly robust quantity for every sample
    drhat = yhat1 - yhat0 + (y - yhat) * (D/Dhat - (1 - D)/(1 - Dhat))
    point = np.mean(drhat)
    var = np.var(drhat)
    stderr = np.sqrt(var / X.shape[0])
    return point, stderr, yhat, Dhat, y - yhat, D - Dhat, drhat

In [6]:
def dr_dirty(X, D, y, modely0_list, modely1_list, modeld_list, *,
             stacker=LinearRegression(), trimming=0.01, nfolds):
    '''
    DML for the Interactive Regression Model setting (Doubly Robust Learning)
    with cross-fitting

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely_list: list of ML models for predicting the outcome y
    modeld_list: list of ML models for predicting the treatment D
    stacker: model used to aggregate predictions of each of the base models
    trimming: threshold below which to trim propensities
    nfolds: the number of folds in cross-fitting

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    Dhat: the cross-fitted predictions for the outcome D
    resy: the outcome residuals
    resD: the treatment residuals
    drhat: the doubly robust quantity for each sample
    '''
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)

    # we will fit a model E[Y| D, X] by fitting a separate model for D==0
    # and a separate model for D==1. We do that for each model type in modely_list
    yhats0, yhats1 = np.zeros((y.shape[0], len(modely0_list))), np.zeros((y.shape[0], len(modely1_list)))
    for train, test in cv.split(X, y):
        for it, modely0 in enumerate(modely0_list):
            yhats0[test, it] = clone(modely0).fit(X.iloc[train][D[train]==0], y[train][D[train]==0]).predict(X.iloc[test])
        for it, modely1 in enumerate(modely1_list):
            yhats1[test, it] = clone(modely1).fit(X.iloc[train][D[train]==1], y[train][D[train]==1]).predict(X.iloc[test])

    # calculate stacking weights for the outcome model for each population
    # and combine the outcome model predictions
    yhat0 = clone(stacker).fit(yhats0[D==0], y[D==0]).predict(yhats0)
    yhat1 = clone(stacker).fit(yhats1[D==1], y[D==1]).predict(yhats1)

    # prediction for observed treatment using the stacked model
    yhat = yhat0 * (1 - D) + yhat1 * D

    # propensity scores
    Dhats = np.array([cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
                     for modeld in modeld_list]).T
    # construct coefficients on each model based on stacker
    Dhat = clone(stacker).fit(Dhats, D).predict(Dhats)
    # trim propensities
    Dhat = np.clip(Dhat, trimming, 1 - trimming)

    # doubly robust quantity for every sample
    drhat = yhat1 - yhat0 + (y - yhat) * (D/Dhat - (1 - D)/(1 - Dhat))
    point = np.mean(drhat)
    var = np.var(drhat)
    stderr = np.sqrt(var / X.shape[0])
    return point, stderr, yhat, Dhat, y - yhat, D - Dhat, drhat

In [7]:
def summary(point, stderr, yhat, Dhat, resy, resD, epsilon, X, D, y, *, name):
    '''
    Convenience summary function that takes the results of the DML function
    and summarizes several estimation quantities and performance metrics.
    '''
    return pd.DataFrame({'estimate': point, # point estimate
                         'stderr': stderr, # standard error
                         'lower': point - 1.96*stderr, # lower end of 95% confidence interval
                         'upper': point + 1.96*stderr, # upper end of 95% confidence interval
                         'rmse y': np.sqrt(np.mean(resy**2)), # RMSE of model that predicts outcome y
                         'rmse D': np.sqrt(np.mean(resD**2)), # RMSE of model that predicts treatment D
                         'accuracy D': np.mean(np.abs(resD) < .5), # binary classification accuracy of model for D
                         }, index=[name])

In [8]:
def create_PLM_table(X, D, y):
    table = pd.DataFrame()
    
    # Double Lasso with crossfitting
    cv = KFold(n_splits=5, shuffle=True, random_state=123)
    lassoy = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
    lassod = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
    result = dml(X, D, y, lassoy, lassod, nfolds=3)
    result = summary(*result, X, D, y, name='double lasso')
    pd.concat([table, result])
    
    
    # Penalized Logistic Regression for D
    cv = KFold(n_splits=5, shuffle=True, random_state=123)
    lassoy = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
    lgrd = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv))
    result = dml(X, D, y, lassoy, lgrd, nfolds=3, classifier=True)
    table = pd.concat([table, summary(*result,  X, D, y, name='lasso/logistic')])
    
    # Random Forest
    rfy = make_pipeline(transformer, RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
    rfd = make_pipeline(transformer, RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
    result = dml(X, D, y, rfy, rfd, nfolds=3, classifier=True)
    table = pd.concat([table, summary(*result,  X, D, y, name='random forest')])

    # Decision Trees
    dtry = make_pipeline(transformer, DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001))
    dtrd = make_pipeline(transformer, DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=.001))
    result = dml(X, D, y, dtry, dtrd, nfolds=3, classifier=True)
    table = pd.concat([table, summary(*result,  X, D, y, name='decision tree')])

    # Boosted Trees
    gbfy = make_pipeline(transformer, GradientBoostingRegressor(max_depth=2, n_iter_no_change=5))
    gbfd = make_pipeline(transformer, GradientBoostingClassifier(max_depth=2, n_iter_no_change=5))
    result = dml(X, D, y, gbfy, gbfd, nfolds=3, classifier=True)
    table = pd.concat([table, summary(*result,  X, D, y, name='boosted forest')])

    # AutoML
    flamly = make_pipeline(transformer, AutoML(time_budget=100, task='regression', early_stop=True,
                                        eval_method='cv', n_splits=3, metric='r2', verbose=3))
    flamld = make_pipeline(transformer, AutoML(time_budget=100, task='classification', early_stop=True,
                                               eval_method='cv', n_splits=3, metric='r2', verbose=3))
    result = dml(X, D, y, flamly, flamld, nfolds=3, classifier=True)
    table = pd.concat([table, summary(*result,  X, D, y, name='automl')])

    # AutoML semi-crossfit
#     flamly = make_pipeline(transformer, AutoML(time_budget=100, task='regression', early_stop=True,
#                                     eval_method='cv', n_splits=3, metric='r2', verbose=0))
#     flamld = make_pipeline(transformer, AutoML(time_budget=100, task='classification', early_stop=True,
#                                            eval_method='cv', n_splits=3, metric='r2', verbose=0))
#     flamly.fit(X, y)
#     besty = make_pipeline(transformer, clone(flamly[-1].best_model_for_estimator(flamly[-1].best_estimator)))
#     flamld.fit(X, D)
#     bestd = make_pipeline(transformer, clone(flamld[-1].best_model_for_estimator(flamld[-1].best_estimator)))
#     result = dml(X, D, y, besty, bestd, nfolds=3, classifier=True)
#     table = pd.concat([table, summary(*result,  X, D, y, name='automl (semi-cfit)')])

    # Stacked semi-crossfit
    result = dml_dirty(X, D, y, [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd],
                   nfolds=3, classifier=True)
    table = pd.concat([table, summary(*result,  X, D, y, name='stacked (semi-cfit)')])

    return table

In [9]:
def create_IRM_table(X, D, y):
    table = pd.DataFrame()
    
    # Lasso/Logistic
    cv = KFold(n_splits=5, shuffle=True, random_state=123)
    lassoy = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
    lgrd = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv))
    result = dr(X, D, y, lassoy, lassoy, lgrd, nfolds=3)
    result = summary(*result, X, D, y, name='lasso/logistic')
    pd.concat([table, result])
    
    
    # Random Forest
    rfy = make_pipeline(transformer, RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
    rfd = make_pipeline(transformer, RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
    result = dr(X, D, y, rfy, rfy, rfd, nfolds=3)
    table = pd.concat([table, summary(*result,  X, D, y, name='random forest')])

    # Decision Trees
    dtry = make_pipeline(transformer, DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001))
    dtrd = make_pipeline(transformer, DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=.001))
    result = dr(X, D, y, dtry, dtry, dtrd, nfolds=3)
    table = pd.concat([table, summary(*result,  X, D, y, name='decision tree')])

    # Boosted Trees
    gbfy = make_pipeline(transformer, GradientBoostingRegressor(max_depth=2, n_iter_no_change=5))
    gbfd = make_pipeline(transformer, GradientBoostingClassifier(max_depth=2, n_iter_no_change=5))
    result = dr(X, D, y, gbfy, gbfy, gbfd, nfolds=3)
    table = pd.concat([table, summary(*result,  X, D, y, name='boosted forest')])

    # AutoML
#     flamly0 = make_pipeline(transformer, AutoML(time_budget=60, task='regression', early_stop=True,
#                                          eval_method='cv', n_splits=3, metric='r2', verbose=0))
#     flamly1 = make_pipeline(transformer, AutoML(time_budget=60, task='regression', early_stop=True,
#                                          eval_method='cv', n_splits=3, metric='r2', verbose=0))
#     flamld = make_pipeline(transformer, AutoML(time_budget=60, task='classification', early_stop=True,
#                                                eval_method='cv', n_splits=3, metric='r2', verbose=0))
#     flamly0.fit(X[D==0], y[D==0])
#     besty0 = make_pipeline(transformer, clone(flamly0[-1].best_model_for_estimator(flamly0[-1].best_estimator)))
#     flamly1.fit(X[D==1], y[D==1])
#     besty1 = make_pipeline(transformer, clone(flamly1[-1].best_model_for_estimator(flamly1[-1].best_estimator)))
#     flamld.fit(X, D)
#     bestd = make_pipeline(transformer, clone(flamld[-1].best_model_for_estimator(flamld[-1].best_estimator)))
#     result = dr(X, D, y, besty0, besty1, bestd, nfolds=3)
#     table = pd.concat([table, summary(*result,  X, D, y, name='automl (semi-cfit)')])

    # Stacked semi-crossfit
    result = dr_dirty(X, D, y, [lassoy, rfy, dtry, gbfy], [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd], nfolds=3)
    table = pd.concat([table, summary(*result,  X, D, y, name='stacked (semi-cfit)')])

    return table

In [10]:
file = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/401k.csv"
data = pd.read_csv(file)

y = data['net_tfa'].values
D = data['e401'].values
D2 = data['p401'].values
D3 = data['a401'].values
X = data.drop(['e401', 'p401', 'a401', 'tw', 'tfa', 'net_tfa', 'tfa_he',
               'hval', 'hmort', 'hequity',
               'nifa', 'net_nifa', 'net_n401', 'ira',
               'dum91', 'icat', 'ecat', 'zhat',
               'i1', 'i2', 'i3', 'i4', 'i5', 'i6', 'i7',
               'a1', 'a2', 'a3', 'a4', 'a5'], axis=1)


In [11]:
# Top 25% of incomes table, PLM

q75 = X['inc'].quantile(.75)
keepIndices = X[ X.inc >= q75 ].index
X_top25 = X.loc[keepIndices]
y_top25 = y[keepIndices]
D_top25 = D[keepIndices]

q25 = X['inc'].quantile(.25)
keepIndices = X[ X.inc <= q25 ].index
X_bottom25 = X.loc[keepIndices]
y_bottom25 = y[keepIndices]
D_bottom25 = D[keepIndices]

transformer = FormulaTransformer("0 + poly(age, degree=6, raw=True) + poly(inc, degree=8, raw=True) "
                                 "+ poly(educ, degree=4, raw=True) + poly(fsize, degree=2, raw=True) "
                                 "+ male + marr + twoearn + db + pira + hown", array=True)

In [12]:
# PLM, top 25% income
create_PLM_table(X_top25, D_top25, y_top25)

,estimate,stderr,lower,upper,rmse y,rmse D,accuracy D
lasso/logistic,17599.136856,3876.385689,10001.420905,25196.852807,91149.933462,0.482720,0.602662
random forest,17519.141514,3968.317264,9741.239677,25297.043351,94305.894884,0.484873,0.603873
decision tree,11358.487068,4075.898655,3369.725705,19347.248431,100253.218084,0.552523,0.529649
boosted forest,18280.803297,3831.720412,10770.631289,25790.975305,95036.967040,0.484491,0.602259
automl,18065.252873,3895.713823,10429.653780,25700.851967,95298.612633,0.487227,0.587334
stacked (semi-cfit),18542.731498,3820.501509,11054.548541,26030.914456,90593.733006,0.481634,0.605083


In [13]:
# IRM, top 25% income
create_IRM_table(X_top25, D_top25, y_top25)

,estimate,stderr,lower,upper,rmse y,rmse D,accuracy D
random forest,17623.308701,3951.243400,9878.871636,25367.745765,96966.742832,0.484563,0.601856
decision tree,-34641.313241,48665.421772,-130025.539915,60742.913432,102482.566021,0.551927,0.528035
boosted forest,17446.345653,3860.792434,9879.192483,25013.498824,95083.010746,0.484808,0.594191
stacked (semi-cfit),17999.370683,3698.084268,10751.125518,25247.615848,90077.014144,0.481897,0.601049


In [14]:
# PLM, bottom 25% income
create_PLM_table(X_bottom25, D_bottom25, y_bottom25)

,estimate,stderr,lower,upper,rmse y,rmse D,accuracy D
lasso/logistic,3765.642212,1059.865913,1688.305022,5842.979402,13374.385873,0.358852,0.846836
random forest,4011.284566,1077.508200,1899.368493,6123.200638,13476.072451,0.344080,0.844015
decision tree,3642.741909,1108.190652,1470.688231,5814.795586,15253.669052,0.376277,0.809351
boosted forest,4187.387308,1112.714411,2006.467063,6368.307553,13474.682391,0.343599,0.846433
automl,4054.580938,1038.486054,2019.148272,6090.013605,13302.179712,0.347103,0.841596
stacked (semi-cfit),3999.230999,1082.822361,1876.899172,6121.562827,13232.706388,0.343444,0.844015


In [15]:
# PLM, bottom 25% income
create_IRM_table(X_bottom25, D_bottom25, y_bottom25)

,estimate,stderr,lower,upper,rmse y,rmse D,accuracy D
random forest,4268.571973,988.706938,2330.706374,6206.437572,13533.730752,0.344661,0.844418
decision tree,10506.846296,6065.384115,-1381.306568,22394.999161,15081.235395,0.376161,0.809351
boosted forest,4781.028185,996.476801,2827.933655,6734.122715,13717.897013,0.344723,0.844821
stacked (semi-cfit),4393.730503,953.923087,2524.041253,6263.419754,13201.983715,0.342357,0.847642


The estimates are much higher for the top 25% of the income distribution compared ot the bottom 25%, indicating heterogeneity on the effect of 401k with respect to income. This is likely because people with higher incomes are more likely to participate in their 401k, and add more to their 401k each month. As a result, their additional accumulated assets through their 401k is higher over the course of their lifetime. 

### Part b

In [16]:
def dml_cf_best(X, D, y, modely_list, modeld_list, *, nfolds, classifier=False):
    '''
    DML for the Partially Linear Model setting with semi-cross-fitting, without stacking

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely: the ML model for predicting the outcome y
    modeld: the ML model for predicting the treatment D
    stacker: model used to aggregate predictions of each of the base models
    nfolds: the number of folds in cross-fitting
    classifier: bool, whether the modeld is a classifier or a regressor

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    bestmodel_yhats: the cross-fitted predictions for the outcome y
    bestmodel_Dhats: the cross-fitted predictions for the treatment D
    resy: the outcome residuals
    resD: the treatment residuals
    epsilon: the final residual-on-residual OLS regression residual
    '''
    # construct out-of-fold predictions for each model
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)
    yhats = np.array([cross_val_predict(modely, X, y, cv=cv, n_jobs=-1) for modely in modely_list]).T
    if classifier:
        Dhats = np.array([cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
                         for modeld in modeld_list]).T
    else:
        Dhats = np.array([cross_val_predict(modeld, X, D, cv=cv, n_jobs=-1) for modeld in modeld_list]).T
        
    # calculate MSEs
    yhats_mses = [np.mean((yhats[:, modely] - y)**2) for modely in range(0, len(modely_list))]
    Dhats_mses = [np.mean((Dhats[:, modeld] - D)**2) for modeld in range(0, len(modeld_list))]
    
    # use yhats and Dhats from best model
    bestmodel_yhats = yhats[:, np.argmin(yhats_mses)]
    bestmodel_Dhats = Dhats[:, np.argmin(Dhats_mses)]
    
    # get ATE
    resy = y - bestmodel_yhats
    resD = D - bestmodel_Dhats
    # go with the stacked residuals
    point = np.mean(resy * resD) / np.mean(resD**2)
    epsilon = resy - point * resD
    var = np.mean(epsilon**2 * resD**2) / np.mean(resD**2)**2
    stderr = np.sqrt(var / X.shape[0])
    
    return point, stderr, bestmodel_yhats, bestmodel_Dhats, resy, resD, epsilon

In [17]:
def dr_cf_best(X, D, y, modely0_list, modely1_list, modeld_list, *, trimming=0.01, nfolds):
    '''
    DML for the Interactive Regression Model setting (Doubly Robust Learning)
    with cross-fitting, without stacking

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely_list: list of ML models for predicting the outcome y
    modeld_list: list of ML models for predicting the treatment D
    stacker: model used to aggregate predictions of each of the base models
    trimming: threshold below which to trim propensities
    nfolds: the number of folds in cross-fitting

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    bestmodel_Dhats: the cross-fitted predictions for the outcome D
    resy: the outcome residuals
    resD: the treatment residuals
    drhat: the doubly robust quantity for each sample
    '''
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)

    # we will fit a model E[Y| D, X] by fitting a separate model for D==0
    # and a separate model for D==1. We do that for each model type in modely_list
    yhats0, yhats1 = np.zeros((y.shape[0], len(modely0_list))), np.zeros((y.shape[0], len(modely1_list)))
    for train, test in cv.split(X, y):
        for it, modely0 in enumerate(modely0_list):
            yhats0[test, it] = clone(modely0).fit(X.iloc[train][D[train]==0], y[train][D[train]==0]).predict(X.iloc[test])
        for it, modely1 in enumerate(modely1_list):
            yhats1[test, it] = clone(modely1).fit(X.iloc[train][D[train]==1], y[train][D[train]==1]).predict(X.iloc[test])

    # propensity scores
    Dhats = np.array([cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
                     for modeld in modeld_list]).T
    
    # calculate MSEs
    y0hats_mses = [np.mean((yhats0[D==0][:, modely] - y[D==0])**2) for modely in range(0, len(modely0_list))]
    y1hats_mses = [np.mean((yhats1[D==1][:, modely] - y[D==1])**2) for modely in range(0, len(modely1_list))]
    Dhats_mses = [np.mean((Dhats[:, modeld] - D)**2) for modeld in range(0, len(modeld_list))]    
    
    # use yhats and Dhats from best model
    bestmodel_y0hats = yhats0[:, np.argmin(y0hats_mses)]
    bestmodel_y1hats = yhats1[:, np.argmin(y1hats_mses)]
    bestmodel_Dhats = Dhats[:, np.argmin(Dhats_mses)]
    
    # prediction for observed treatment using the best model
    yhat = bestmodel_y0hats * (1 - D) + bestmodel_y1hats * D 
    

    # trim propensities
    bestmodel_Dhats = np.clip(bestmodel_Dhats, trimming, 1 - trimming)

    # doubly robust quantity for every sample
    drhat = bestmodel_y1hats - bestmodel_y0hats + (y - yhat) * (D/bestmodel_Dhats - (1 - D)/(1 - bestmodel_Dhats))
    point = np.mean(drhat)
    var = np.var(drhat)
    stderr = np.sqrt(var / X.shape[0])
    return point, stderr, yhat, bestmodel_Dhats, y - yhat, D - bestmodel_Dhats, drhat

In [18]:
# DML table on full data, including best model function implemented above

cv = KFold(n_splits=5, shuffle=True, random_state=123)
lassoy = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
lassod = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
result = dml(X, D, y, lassoy, lassod, nfolds=3)
table = summary(*result, X, D, y, name='double lasso')

cv = KFold(n_splits=5, shuffle=True, random_state=123)
lassoy = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
lgrd = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv))
result = dml(X, D, y, lassoy, lgrd, nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='lasso/logistic')])

rfy = make_pipeline(transformer, RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
rfd = make_pipeline(transformer, RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
result = dml(X, D, y, rfy, rfd, nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='random forest')])

dtry = make_pipeline(transformer, DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001))
dtrd = make_pipeline(transformer, DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=.001))
result = dml(X, D, y, dtry, dtrd, nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='decision tree')])

gbfy = make_pipeline(transformer, GradientBoostingRegressor(max_depth=2, n_iter_no_change=5))
gbfd = make_pipeline(transformer, GradientBoostingClassifier(max_depth=2, n_iter_no_change=5))
result = dml(X, D, y, gbfy, gbfd, nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='boosted forest')])

flamly = make_pipeline(transformer, AutoML(time_budget=100, task='regression', early_stop=True,
                                    eval_method='cv', n_splits=3, metric='r2', verbose=3))
flamld = make_pipeline(transformer, AutoML(time_budget=100, task='classification', early_stop=True,
                                           eval_method='cv', n_splits=3, metric='r2', verbose=3))
result = dml(X, D, y, flamly, flamld, nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='AutoML')])

# flamly = make_pipeline(transformer, AutoML(time_budget=100, task='regression', early_stop=True,
#                                     eval_method='cv', n_splits=3, metric='r2', verbose=0))
# flamld = make_pipeline(transformer, AutoML(time_budget=100, task='classification', early_stop=True,
#                                            eval_method='cv', n_splits=3, metric='r2', verbose=0))
# flamly.fit(X, y)
# besty = make_pipeline(transformer, clone(flamly[-1].best_model_for_estimator(flamly[-1].best_estimator)))
# flamld.fit(X, D)
# bestd = make_pipeline(transformer, clone(flamld[-1].best_model_for_estimator(flamld[-1].best_estimator)))
# result = dml(X, D, y, besty, bestd, nfolds=3, classifier=True)
# table = pd.concat([table, summary(*result,  X, D, y, name='automl (semi-cfit)')])

result = dml_dirty(X, D, y, [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd],
                   nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='stacked (semi-cfit)')])

result = dml_cf_best(X, D, y, [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd],
                   nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='best (semi-cfit)')])

table

,estimate,stderr,lower,upper,rmse y,rmse D,accuracy D
double lasso,8914.299159,1289.335778,6387.201034,11441.397284,54111.994234,0.443826,0.688048
lasso/logistic,8494.171826,1292.180346,5961.498348,11026.845303,54111.994234,0.444247,0.686233
random forest,8948.523137,1365.770525,6271.612907,11625.433367,55131.431552,0.444534,0.691377
decision tree,9354.674857,1478.080231,6457.637605,12251.712109,59843.566636,0.447080,0.678467
boosted forest,9287.725203,1393.387694,6556.685323,12018.765083,56237.867900,0.444240,0.690872
AutoML,9282.807506,1367.186950,6603.121083,11962.493929,55597.261019,0.444929,0.689259
stacked (semi-cfit),8958.135007,1307.606453,6395.226360,11521.043655,53897.076928,0.443180,0.691881
best (semi-cfit),8889.630317,1319.102890,6304.188653,11475.071981,54111.994234,0.443922,0.692789


In [19]:
# DR table on full data, including best model function implemented above

cv = KFold(n_splits=5, shuffle=True, random_state=123)
lassoy = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
lgrd = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv))
result = dr(X, D, y, lassoy, lassoy, lgrd, nfolds=3)
table = summary(*result, X, D, y, name='lasso/logistic')

rfy = make_pipeline(transformer, RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
rfd = make_pipeline(transformer, RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
result = dr(X, D, y, rfy, rfy, rfd, nfolds=3)
table = pd.concat([table, summary(*result,  X, D, y, name='random forest')])

dtry = make_pipeline(transformer, DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001))
dtrd = make_pipeline(transformer, DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=.001))
result = dr(X, D, y, dtry, dtry, dtrd, nfolds=3)
table = pd.concat([table, summary(*result,  X, D, y, name='decision tree')])

gbfy = make_pipeline(transformer, GradientBoostingRegressor(max_depth=2, n_iter_no_change=5))
gbfd = make_pipeline(transformer, GradientBoostingClassifier(max_depth=2, n_iter_no_change=5))
result = dr(X, D, y, gbfy, gbfy, gbfd, nfolds=3)
table = pd.concat([table, summary(*result,  X, D, y, name='boosted forest')])

# flamly0 = make_pipeline(transformer, AutoML(time_budget=60, task='regression', early_stop=True,
#                                      eval_method='cv', n_splits=3, metric='r2', verbose=0))
# flamly1 = make_pipeline(transformer, AutoML(time_budget=60, task='regression', early_stop=True,
#                                      eval_method='cv', n_splits=3, metric='r2', verbose=0))
# flamld = make_pipeline(transformer, AutoML(time_budget=60, task='classification', early_stop=True,
#                                            eval_method='cv', n_splits=3, metric='r2', verbose=0))
# flamly0.fit(X[D==0], y[D==0])
# besty0 = make_pipeline(transformer, clone(flamly0[-1].best_model_for_estimator(flamly0[-1].best_estimator)))
# flamly1.fit(X[D==1], y[D==1])
# besty1 = make_pipeline(transformer, clone(flamly1[-1].best_model_for_estimator(flamly1[-1].best_estimator)))
# flamld.fit(X, D)
# bestd = make_pipeline(transformer, clone(flamld[-1].best_model_for_estimator(flamld[-1].best_estimator)))
# result = dr(X, D, y, besty0, besty1, bestd, nfolds=3)
# table = pd.concat([table, summary(*result,  X, D, y, name='automl (semi-cfit)')])

result = dr_dirty(X, D, y, [lassoy, rfy, dtry, gbfy], [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd], nfolds=3)
table = pd.concat([table, summary(*result,  X, D, y, name='stacked (semi-cfit)')])

result = dr_cf_best(X, D, y, [lassoy, rfy, dtry, gbfy], [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd], nfolds=3)
table = pd.concat([table, summary(*result,  X, D, y, name='best (semi-cfit)')])

table

,estimate,stderr,lower,upper,rmse y,rmse D,accuracy D
lasso/logistic,2535.427991,5434.603246,-8116.394371,13187.250352,54023.763725,0.444247,0.686233
random forest,7819.960558,1162.348040,5541.758401,10098.162716,55489.789340,0.444623,0.691074
decision tree,7077.394356,1290.400732,4548.208921,9606.579790,60264.954815,0.447080,0.678467
boosted forest,8553.342176,1158.530304,6282.622781,10824.061572,54960.051588,0.444021,0.691276
stacked (semi-cfit),7616.709741,1139.060553,5384.151058,9849.268424,53836.232349,0.443248,0.689864
best (semi-cfit),7873.570587,1129.950106,5658.868379,10088.272795,54023.763725,0.444186,0.692789


### Part c

In [20]:
# Using EconML

from econml.dml import LinearDML
from econml.dr import LinearDRLearner
from econml.utilities import SeparateModel


In [21]:
W = StandardScaler().fit_transform(transformer.fit_transform(X))

In [22]:
# Random Forest

rfml = LinearDML(model_y=RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001), 
                 model_t=RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001),
                 cv=3, discrete_treatment=True, random_state=123).fit(y, D, W=W)
rfml.summary()

Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,8184.146,1350.077,6.062,0.0,5538.044,10830.248


In [23]:
# Gradient Boosted Tree

gbt = LinearDML(model_y=GradientBoostingRegressor(max_depth=5, n_iter_no_change=10), 
                model_t=GradientBoostingRegressor(max_depth=5, n_iter_no_change=10),
                cv=3, discrete_treatment=True, random_state=123).fit(y, D, W=W)
gbt.summary()

Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,8713.87,1426.973,6.107,0.0,5917.055,11510.685


In [24]:
# Decision Tree


dt = LinearDML(model_y=DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001), 
               model_t=DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001),
               cv=3, discrete_treatment=True, random_state=123).fit(y, D, W=W)

dt.summary()

Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,8598.567,1417.949,6.064,0.0,5819.438,11377.697


In [25]:
# Doubly Robust Random Forest

drrf = LinearDRLearner(
            model_regression=SeparateModel(
                RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001), 
                RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001)),
            model_propensity=RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001),
            cv=3, min_propensity=0.01, random_state=123).fit(y, D, W=W)

drrf.summary(T=1)

Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,7508.136,1137.421,6.601,0.0,5278.832,9737.44


In [26]:
# Doubly Robust Gradient Boosted Tree

drgbt = LinearDRLearner(
            model_regression=SeparateModel(
                GradientBoostingRegressor(max_depth=5, n_iter_no_change=10), 
                GradientBoostingRegressor(max_depth=5, n_iter_no_change=10)),
            model_propensity=GradientBoostingClassifier(max_depth=5, n_iter_no_change=10),
            cv=3, min_propensity=0.01, random_state=123).fit(y, D, W=W)

drgbt.summary(T=1)

Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,7757.003,1217.821,6.37,0.0,5370.118,10143.888


In [27]:
# Using DoubleML

import doubleml as dml
from doubleml import DoubleMLData
dml_data = DoubleMLData.from_arrays(W, y, D)

In [28]:
# Random Forest

rf = dml.DoubleMLPLR(dml_data, 
                    RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001), 
                    RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001), 
                    n_folds=3)
print(rf.fit())

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26']
Instrument variable(s): None
No. Observations: 9915

------------------ Score & algorithm ------------------
Score function: partialling out
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRegressor(ccp_alpha=0.001, min_samples_leaf=10)
Learner ml_m: RandomForestRegressor(ccp_alpha=0.001, min_samples_leaf=10)
Out-of-sample Performance:
Learner ml_l RMSE: [[55301.91001332]]
Learner ml_m RMSE: [[0.44559897]]

------------------ Resampling        ------------------
No. folds: 3
No. repeated sample splits: 1
Apply cross-fitting: True

------------------ Fit summary       ------------------
   

In [29]:
# Gradient Boosted Tree

gbt = dml.DoubleMLPLR(dml_data, 
                    GradientBoostingRegressor(max_depth=5, n_iter_no_change=10), 
                    GradientBoostingRegressor(max_depth=5, n_iter_no_change=10), 
                    n_folds=3)
print(gbt.fit())

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26']
Instrument variable(s): None
No. Observations: 9915

------------------ Score & algorithm ------------------
Score function: partialling out
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_l: GradientBoostingRegressor(max_depth=5, n_iter_no_change=10)
Learner ml_m: GradientBoostingRegressor(max_depth=5, n_iter_no_change=10)
Out-of-sample Performance:
Learner ml_l RMSE: [[60896.96883926]]
Learner ml_m RMSE: [[0.44676685]]

------------------ Resampling        ------------------
No. folds: 3
No. repeated sample splits: 1
Apply cross-fitting: True

------------------ Fit summary       ------------------
   

In [30]:
# Decision Tree

dt = dml.DoubleMLPLR(dml_data, 
                    DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001), 
                    DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001), 
                    n_folds=3)
print(dt.fit())

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26']
Instrument variable(s): None
No. Observations: 9915

------------------ Score & algorithm ------------------
Score function: partialling out
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_l: DecisionTreeRegressor(ccp_alpha=0.001, min_samples_leaf=10)
Learner ml_m: DecisionTreeRegressor(ccp_alpha=0.001, min_samples_leaf=10)
Out-of-sample Performance:
Learner ml_l RMSE: [[60624.47696777]]
Learner ml_m RMSE: [[0.44708798]]

------------------ Resampling        ------------------
No. folds: 3
No. repeated sample splits: 1
Apply cross-fitting: True

------------------ Fit summary       ------------------
   

In [31]:
# Doubly Robust Random Forest

drrf = dml.DoubleMLIRM(dml_data, 
                       RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001), 
                       RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001), 
                       n_folds=3)

print(drrf.fit())

================== DoubleMLIRM Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26']
Instrument variable(s): None
No. Observations: 9915

------------------ Score & algorithm ------------------
Score function: ATE
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_g: RandomForestRegressor(ccp_alpha=0.001, min_samples_leaf=10)
Learner ml_m: RandomForestClassifier(ccp_alpha=0.001, min_samples_leaf=10)
Out-of-sample Performance:
Learner ml_g0 RMSE: [[48043.1981954]]
Learner ml_g1 RMSE: [[65238.56900373]]
Learner ml_m RMSE: [[0.44509545]]

------------------ Resampling        ------------------
No. folds: 3
No. repeated sample splits: 1
Apply cross-fitting: True

------------------ Fit summary 

In [32]:
# Doubly Robust Gradient Boosted Tree

drgbt = dml.DoubleMLIRM(dml_data, 
                       GradientBoostingRegressor(max_depth=5, n_iter_no_change=10), 
                       GradientBoostingClassifier(max_depth=5, n_iter_no_change=10), 
                       n_folds=3)

print(drgbt.fit())

================== DoubleMLIRM Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26']
Instrument variable(s): None
No. Observations: 9915

------------------ Score & algorithm ------------------
Score function: ATE
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_g: GradientBoostingRegressor(max_depth=5, n_iter_no_change=10)
Learner ml_m: GradientBoostingClassifier(max_depth=5, n_iter_no_change=10)
Out-of-sample Performance:
Learner ml_g0 RMSE: [[51636.64732086]]
Learner ml_g1 RMSE: [[72580.5371667]]
Learner ml_m RMSE: [[0.44785273]]

------------------ Resampling        ------------------
No. folds: 3
No. repeated sample splits: 1
Apply cross-fitting: True

------------------ Fit summary 

In [33]:
# Doubly Robust Decision Tree


drdt = dml.DoubleMLIRM(dml_data, 
                       DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001), 
                       DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=.001), 
                       n_folds=3)

print(drdt.fit())

================== DoubleMLIRM Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26']
Instrument variable(s): None
No. Observations: 9915

------------------ Score & algorithm ------------------
Score function: ATE
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_g: DecisionTreeRegressor(ccp_alpha=0.001, min_samples_leaf=10)
Learner ml_m: DecisionTreeClassifier(ccp_alpha=0.001, min_samples_leaf=10)
Out-of-sample Performance:
Learner ml_g0 RMSE: [[52806.09384908]]
Learner ml_g1 RMSE: [[70325.56721675]]
Learner ml_m RMSE: [[0.44661613]]

------------------ Resampling        ------------------
No. folds: 3
No. repeated sample splits: 1
Apply cross-fitting: True

------------------ Fit summary

I was able to implement all the different model types with both PLM and IRM models, using both EconML and DoubleML. 

### Part d

In [34]:
class semisynth:
    
    def fit(self, X, D, y):
        self.X_ = X.copy()
        self.est0_ = make_pipeline(transformer,
                                   RandomForestRegressor(min_samples_leaf=20, ccp_alpha=0.001)).fit(X[D==0], y[D==0])
        self.res0_ = y[D==0] - self.est0_.predict(X[D==0])
        self.res0_ -= np.mean(self.res0_)
        self.est1_ = make_pipeline(transformer,
                                   RandomForestRegressor(min_samples_leaf=20, ccp_alpha=0.001)).fit(X[D==1], y[D==1])
        self.res1_ = y[D==1] - self.est1_.predict(X[D==1])
        self.res1_ -= np.mean(self.res1_)
        self.prop_ = make_pipeline(transformer,
                                   RandomForestClassifier(min_samples_leaf=20, ccp_alpha=0.001)).fit(X, D)
        return self

    def generate_data(self, n):
        X = self.X_.iloc[np.random.choice(self.X_.shape[0], n, replace=True)]
        D = np.random.binomial(1, self.prop_.predict_proba(X)[:, 1])
        y0 = self.est0_.predict(X) + self.res0_[np.random.choice(self.res0_.shape[0], n, replace=True)]
        y1 = self.est1_.predict(X) + self.res1_[np.random.choice(self.res1_.shape[0], n, replace=True)]
        y = y0 * (1 - D) + y1 * D
        return X, D, y, y1, y0
    
    def y_cef(self, X, D):
        return self.est1_.predict(X) * D + self.est0_.predict(X) * (1 - D)
    
    def D_cef(self, X):
        return self.prop_.predict_proba(X)[:, 1]

    @property
    def true_ate(self):
        return np.mean(self.est1_.predict(self.X_) - self.est0_.predict(self.X_))

In [35]:
synth = semisynth()

synth.fit(X, D, y)

synth.true_ate

In [36]:
def summary(point, stderr, yhat, Dhat, resy, resD, epsilon, X, D, y, *, name):
    '''
    Convenience summary function that takes the results of the DML function
    and summarizes several estimation quantities and performance metrics.
    '''
    return pd.DataFrame({'estimate': point, # point estimate
                         'stderr': stderr, # standard error
                         'lower': point - 1.96*stderr, # lower end of 95% confidence interval
                         'upper': point + 1.96*stderr, # upper end of 95% confidence interval
                         'rmse y': np.sqrt(np.mean(resy**2)), # RMSE of model that predicts outcome y
                         'rmse D': np.sqrt(np.mean(resD**2)), # RMSE of model that predicts treatment D
                         'estimate error': np.abs(point - synth.true_ate), # binary classification accuracy of model for D
                         'rmse E[y|D,X]': np.sqrt(np.mean((yhat - synth.y_cef(X, D))**2)), # RMSE wrt true function E[y|X]
                         'rmse E[D|X]': np.sqrt(np.mean((Dhat - synth.D_cef(X))**2)), # RMSE wrt true funciton E[D|X]
                         'coverage': (point - 1.96*stderr <= synth.true_ate) & (point + 1.96*stderr >= synth.true_ate)
                         }, index=[name])

In [39]:
# n=1000

X, D, y, y1, y0 = synth.generate_data(1000)

result = dml(X, D, y, lassoy, lassod, nfolds=3)
table = summary(*result, X, D, y, name='double lasso')

result = dml(X, D, y, lassoy, lgrd, nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='dml lasso/logistic')])

result = dml(X, D, y, rfy, rfd, nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='dml random forest')])

result = dml(X, D, y, dtry, dtrd, nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='dml decision tree')])

result = dml(X, D, y, gbfy, gbfd, nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='dml boosted forest')])

result = dml(X, D, y, flamly, flamld, nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='dml AutoML')])

result = dml_dirty(X, D, y, [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd],
                   nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='dml stacked (semi-cfit)')])

result = dml_cf_best(X, D, y, [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd],
                   nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='dml best (semi-cfit)')])

result = dr(X, D, y, lassoy, lassoy, lgrd, nfolds=3)
table = pd.concat([table, summary(*result,  X, D, y, name='dr lasso/logistic')])

result = dr(X, D, y, rfy, rfy, rfd, nfolds=3)
table = pd.concat([table, summary(*result,  X, D, y, name='dr random forest')])

result = dr(X, D, y, dtry, dtry, dtrd, nfolds=3)
table = pd.concat([table, summary(*result,  X, D, y, name='dr decision tree')])

result = dr(X, D, y, gbfy, gbfy, gbfd, nfolds=3)
table = pd.concat([table, summary(*result,  X, D, y, name='dr boosted forest')])

result = dr_dirty(X, D, y, [lassoy, rfy, dtry, gbfy], [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd], nfolds=3)
table = pd.concat([table, summary(*result,  X, D, y, name='dr stacked (semi-cfit)')])

result = dr_cf_best(X, D, y, [lassoy, rfy, dtry, gbfy], [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd], nfolds=3)
table = pd.concat([table, summary(*result,  X, D, y, name='dr best (semi-cfit)')])

table

,estimate,stderr,lower,upper,rmse y,rmse D,estimate error,"rmse E[y|D,X]",rmse E[D|X],coverage
double lasso,8521.806337,4491.532017,-281.596417,17325.209091,60010.633351,0.464844,1064.921306,17349.446485,0.082055,True
dml lasso/logistic,7967.404289,4396.741342,-650.208740,16585.017319,60010.633351,0.469868,510.519258,17349.446485,0.092060,True
dml random forest,10051.771391,4480.760019,1269.481754,18834.061028,60149.640435,0.466639,2594.886360,19070.538696,0.092050,True
dml decision tree,6457.600634,3966.755975,-1317.241076,14232.442344,64720.488964,0.527258,999.284397,28490.227729,0.260645,True
dml boosted forest,10284.526556,4379.393141,1700.915999,18868.137112,59813.118428,0.465398,2827.641524,18120.169985,0.083616,True
dml AutoML,10126.908159,4551.842048,1205.297745,19048.518572,58989.856502,0.465666,2670.023128,15998.845375,0.099215,True
dml stacked (semi-cfit),8341.029613,4401.579046,-286.065317,16968.124543,59525.184686,0.464504,884.144582,16221.954129,0.077575,True
dml best (semi-cfit),8677.319864,4565.626541,-271.308156,17625.947884,60004.162239,0.464585,1220.434833,18659.878279,0.086959,True
dr lasso/logistic,7531.580349,4685.892610,-1652.769167,16715.929866,59472.131695,0.469868,74.695318,16550.461067,0.092060,True
dr random forest,9854.949749,5111.070684,-162.748791,19872.648289,60235.138260,0.465427,2398.064718,19800.781052,0.090841,True


In [40]:
# n=10000

X, D, y, y1, y0 = synth.generate_data(10000)

result = dml(X, D, y, lassoy, lassod, nfolds=3)
table = summary(*result, X, D, y, name='double lasso')

result = dml(X, D, y, lassoy, lgrd, nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='dml lasso/logistic')])

result = dml(X, D, y, rfy, rfd, nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='dml random forest')])

result = dml(X, D, y, dtry, dtrd, nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='dml decision tree')])

result = dml(X, D, y, gbfy, gbfd, nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='dml boosted forest')])

result = dml(X, D, y, flamly, flamld, nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='dml AutoML')])

result = dml_dirty(X, D, y, [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd],
                   nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='dml stacked (semi-cfit)')])

result = dml_cf_best(X, D, y, [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd],
                   nfolds=3, classifier=True)
table = pd.concat([table, summary(*result,  X, D, y, name='dml best (semi-cfit)')])

result = dr(X, D, y, lassoy, lassoy, lgrd, nfolds=3)
table = pd.concat([table, summary(*result,  X, D, y, name='dr lasso/logistic')])

result = dr(X, D, y, rfy, rfy, rfd, nfolds=3)
table = pd.concat([table, summary(*result,  X, D, y, name='dr random forest')])

result = dr(X, D, y, dtry, dtry, dtrd, nfolds=3)
table = pd.concat([table, summary(*result,  X, D, y, name='dr decision tree')])

result = dr(X, D, y, gbfy, gbfy, gbfd, nfolds=3)
table = pd.concat([table, summary(*result,  X, D, y, name='dr boosted forest')])

result = dr_dirty(X, D, y, [lassoy, rfy, dtry, gbfy], [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd], nfolds=3)
table = pd.concat([table, summary(*result,  X, D, y, name='dr stacked (semi-cfit)')])

result = dr_cf_best(X, D, y, [lassoy, rfy, dtry, gbfy], [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd], nfolds=3)
table = pd.concat([table, summary(*result,  X, D, y, name='dr best (semi-cfit)')])

table

,estimate,stderr,lower,upper,rmse y,rmse D,estimate error,"rmse E[y|D,X]",rmse E[D|X],coverage
double lasso,6637.200551,1226.208616,4233.831665,9040.569437,49978.265814,0.454211,819.684480,12916.842988,0.040850,True
dml lasso/logistic,6588.965797,1230.213664,4177.747015,9000.184579,49978.265814,0.454332,867.919234,12916.842988,0.041739,True
dml random forest,6865.903096,1202.523492,4508.957051,9222.849141,49687.982284,0.454950,590.981935,11887.160973,0.038562,True
dml decision tree,7404.736535,1256.197196,4942.590031,9866.883038,52932.440847,0.455662,52.148496,21689.951367,0.046165,True
dml boosted forest,6908.600861,1201.663155,4553.341078,9263.860645,49562.387518,0.454698,548.284170,11616.432694,0.037088,True
dml AutoML,7414.440810,1192.857414,5076.440279,9752.441341,49566.334685,0.454904,42.444221,11685.244867,0.039175,True
dml stacked (semi-cfit),6830.798747,1208.089602,4462.943128,9198.654366,49336.497727,0.453754,626.086284,10448.087543,0.028796,True
dml best (semi-cfit),6874.774509,1219.453363,4484.645917,9264.903101,49568.197334,0.454332,582.110522,11439.807488,0.041739,True
dr lasso/logistic,5965.554611,1235.183728,3544.594503,8386.514719,49870.226450,0.454332,1491.330420,12194.547377,0.041739,True
dr random forest,6624.568264,1179.877380,4312.008600,8937.127928,49481.623919,0.454750,832.316767,9891.794112,0.038770,True


The autoML and stacked models work well compared to the others. Their estimates are similar and standard errors are close to if not lower than others. This is true for both n=1000 and n=10000. 